# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [1]:
from datapackage import Package

In [2]:
# URL pointing to the raw datapackage.json file of the scenario
url = "https://raw.githubusercontent.com/premise-community-scenarios/energy-perspective-2050-switzerland/main/datapackage.json"
switzerland_2050 = Package(url)

The datapackage has 3 resources: scenario data, inventories and a configuration file.

In [3]:
for resource in switzerland_2050.resources:
    print(resource.name)

scenario_data
inventories
config


or

In [4]:
switzerland_2050.resource_names

['scenario_data', 'inventories', 'config']

And you can directly read them (or look the resources up directly from the repo):

In [5]:
import pandas as pd
data = switzerland_2050.get_resource("scenario_data").read()
pd.DataFrame(data).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Run-of-river hydro,PJ/year,64.002,63.994,63.928,63.119,61.774,59.235,58.097,58.097,58.097,58.097,58.097,58.097
1,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Reservoir,PJ/year,76.367,79.679,81.45,79.47,79.556,81.238,82.102,82.102,82.102,82.102,82.102,82.102
2,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Nuclear|Boiling water r...,PJ/year,37.30202,28.321,15.165,0,0,0,0,0,0,0,0,0
3,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Nuclear|Pressure water ...,PJ/year,42.06398,31.936,17.102,0,0,0,0,0,0,0,0,0
4,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Conventional|Waste-to-E...,PJ/year,6.767,6.883,6.998,7.114,7.229,7.345,7.461,7.461,7.461,7.461,7.461,7.461


The datapackage can also be locally stored (i.e., on your computer), in which case you only
need to provide the filepath to the `datapackage.json` file.

Once the datapackage is loaded, you just need to pass it (in a list) to the `NewDatabase` instance of `premise`.

In [6]:
#check version of premise
import premise
v=str(premise.__version__).replace(',','.')
for character in ')( ':
    v = v.replace(character, '')
    
pv="PREMISE v"+v
print(pv)

PREMISE v1.3.6


In [7]:
from premise import *
import brightway2 as bw

In [8]:
if 'ei38-teaching' not in bw.projects:
    bw.restore_project_directory("/srv/data/projects/ecoinvent38.tar.gz")

In [9]:
bw.projects.set_current("ei38-teaching")

In [10]:
#bw.projects.copy_project("moeschpond1")

In [11]:
bw.projects.set_current("moeschpond1")

In [12]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [13]:
%%time
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2050},
    {"model": "image", "pathway":"SSP2-RCP26", "year": 2050},
    {"model": "image", "pathway":"SSP2-RCP19", "year": 2050},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2050},
    #{"model": "remind", "pathway":"SSP2-NDC", "year": 2050},
    #{"model": "remind", "pathway":"SSP2-NPi", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2050},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ei 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            switzerland_2050,
        ]
)


premise v.(1, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

To integrate the projections of the user-generated scenario, call `update_external_scenario()`.

In [14]:
%%time #should take 10 minutes
ndb.update_electricity()
#ndb.update_all()#update all sectors that PREMISE can do
ndb.update_external_scenario()#also update the external switzerland2050 scenario


/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Log of changes in photovoltaics efficiencies saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Create biomass markets.
Empty old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Log of deleted electricity markets saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Log of created electricity markets saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Done!
Adjust efficiency of power plants...
Log of changes in power plants efficiencies saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Rescale inventories and emissions for Biomass IGCC
Rescale inventories and emissions for Coal IGCC CCS
Rescale inventories and emissions for Biomass IGCC CCS
Rescale inventories and emissions for Ga

You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).

In [19]:
%%time
ndb.write_superstructure_db_to_brightway()

Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 3.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 4.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 5.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 6.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Building superstructure database...
Dropped 0 duplicate(s).
Scenario difference file exported to /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/export/scenario diff files!
Done!


AssertionError: This database already exists

In [22]:
#call from linux terminal to move the scenario diff file to our local github folder. The filename includes the date of generation. This is currently not automized, so remeber to change the filename according to the output from write_superstructure_db_to_brightway() 
!cp /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/export/scenario\ diff\ files/scenario_diff_super_db_2022-10-26.xlsx /home/jupyter-simb/moeschpond/

In [34]:
bw.databases
len(bw.Database("super_db_2022-10-26"))

25319

In [28]:
bw.backup_project_directory("moeschpond1")

Creating project backup archive - this could take a few minutes...


In [32]:
#DO NOT PUT ECOINVENT ON PUBLIC GITHUB
#copy the file to shared directory on server.
!cp ~/brightway2-project-moeschpond1-backup.26-October-2022-03-51PM.tar.gz /srv/scratch/u.2/

In [26]:
bw.restore_project_directory("/srv/scratch/u.2/brightway2-project-moeschpond1-backup.26-October-2022-03-51PM.tar.gz")

Restoring project backup archive - this could take a few minutes...


'moeschpond1'